# Hypothesis

We can predict how many medals a country will win at the Olympics by using historical data.

# The Data

A dataset of how many medals each country won at each Olympics.  Other data would also be nice (number of athletes, etc).

In [13]:
import pandas as pd

In [14]:
teams = pd.read_csv(r"D:\Data centr\Exel_data\int data\teams.csv")

In [15]:
teams

,team,country,year,events,athletes,age,height,weight,medals,prev_medals,prev_3_medals
0,AFG,Afghanistan,1964,8,8,22.0,161.0,64.2,0,0.0,0.0
1,AFG,Afghanistan,1968,5,5,23.2,170.2,70.0,0,0.0,0.0
2,AFG,Afghanistan,1972,8,8,29.0,168.3,63.8,0,0.0,0.0
3,AFG,Afghanistan,1980,11,11,23.6,168.4,63.2,0,0.0,0.0
4,AFG,Afghanistan,2004,5,5,18.6,170.8,64.8,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2139,ZIM,Zimbabwe,2000,19,26,25.0,179.0,71.1,0,0.0,0.0
2140,ZIM,Zimbabwe,2004,11,14,25.1,177.8,70.5,3,0.0,0.0
2141,ZIM,Zimbabwe,2008,15,16,26.1,171.9,63.7,4,3.0,1.0
2142,ZIM,Zimbabwe,2012,8,9,27.3,174.4,65.2,0,4.0,2.3


In [22]:
data = {
    'team': ['USA', 'CHN', 'RUS', 'AFG', 'IND'],
    'medals': ['46', '38', '27', 'AFG', '24'],
    'score': ['100', '200', '150', 'AFG', '300']
}

In [23]:
teams = pd.DataFrame(data)

In [25]:
print(teams)

  team medals score
0  USA     46   100
1  CHN     38   200
2  RUS     27   150
3  AFG    AFG   AFG
4  IND     24   300


In [26]:
teams['medals'] = pd.to_numeric(teams['medals'], errors='coerce')
teams['score'] = pd.to_numeric(teams['score'], errors='coerce')

In [28]:
teams_cleaned = teams.dropna()

In [29]:
print(teams_cleaned.corr()['medals'])

ValueError: could not convert string to float: 'USA'

In [ ]:
import seaborn as sns

In [ ]:
sns.lmplot(x='athletes',y='medals',data=teams,fit_reg=True, ci=None) 

In [ ]:
sns.lmplot(x='age', y='medals', data=teams, fit_reg=True, ci=None) 

In [ ]:
teams.plot.hist(y="medals")

In [ ]:
teams[teams.isnull().any(axis=1)].head(20)

In [ ]:
teams = teams.dropna()

In [ ]:
teams.shape

In [ ]:
train = teams[teams["year"] < 2012].copy()
test = teams[teams["year"] >= 2012].copy()

In [ ]:
# About 80% of the data
train.shape

In [ ]:
# About 20% of the data
test.shape

# Accuracy Metric

We'll use mean squared error.  This is a good default regression accuracy metric.  It's the average of squared differences between the actual results and your predictions.

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

In [ ]:
predictors = ["athletes", "prev_medals"]

In [ ]:
reg.fit(train[predictors], train["medals"])

In [ ]:
predictions = reg.predict(test[predictors])

In [ ]:
predictions.shape

In [ ]:
test["predictions"] = predictions

In [ ]:
test.loc[test["predictions"] < 0, "predictions"] = 0

In [ ]:
test["predictions"] = test["predictions"].round()

In [ ]:
from sklearn.metrics import mean_absolute_error

error = mean_absolute_error(test["medals"], test["predictions"])
error

In [ ]:
teams.describe()["medals"]

In [ ]:
test["predictions"] = predictions

In [ ]:
test[test["team"] == "USA"]

In [ ]:
test[test["team"] == "IND"]

In [ ]:
errors = (test["medals"] - predictions).abs()

In [ ]:
error_by_team = errors.groupby(test["team"]).mean()
medals_by_team = test["medals"].groupby(test["team"]).mean()
error_ratio =  error_by_team / medals_by_team 

In [ ]:
import numpy as np
error_ratio = error_ratio[np.isfinite(error_ratio)]

In [ ]:
error_ratio.plot.hist()

In [ ]:
error_ratio.sort_values()

# Next steps

This model works well for countries which have a high medal count, and compete in a stable number of events annually.  For countries that get fewer medals, you'd want to build this model in a different way.

Some potential next steps:

* Add in some more predictors to the model, like `height`, `athletes`, or `age`.
* Go back to the original, athlete-level data (`athlete_events.csv`), and try to compute some additional variables, like total number of years competing in the Olympics.
* For countries with low medal counts, you can try modelling if individual athletes will win their event.  You can build event-specific models to predict if athletes will win their events.  Then you can add up the predicted medals for each athlete from each country.  This will give you the total predicted medal count for that country.